In [1]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 8.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=e13a8f13d77e0a84a863da752358861cddccd26c6fcc7d26d98c340bd46d9e4a
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.fftpack
import random
%matplotlib inline

In [3]:
def StockStatsCheck(df):

  dfMean = (df['High']+df['Low'])/2

  meanArr =  dfMean.groupby(pd.Grouper(freq='1d')).mean().dropna()
  stdArr  =  dfMean.groupby(pd.Grouper(freq='1d')).std().dropna()

  #print(meanArr)
  #print(stdArr)

  meanArr = np.array(meanArr)
  stdArr = np.array(stdArr)

  #print(abs(1-meanArr[:-1]/meanArr[1:]))
  #print(abs(1-meanArr[:-1]/meanArr[1:]) < MEAN_FLUCT * ARB_VAL)
  #print(stdArr/meanArr)
  #print(stdArr/meanArr >  STD_TO_ARB * ARB_VAL)

  c1_mean_val = 0*abs(1-meanArr[:-1]/meanArr[1:])
  c2_std_val = (STD_TO_ARB*stdArr)/meanArr

  c1_mean = c1_mean_val < MEAN_FLUCT * ARB_VAL
  c2_std = c2_std_val > ARB_VAL

  #print(c1,c2)
  c_tot = all(c1_mean) and all(c2_std) and (len(c1_mean)>0) and (len(c2_std)>0)
  if (c_tot): print (c1_mean,c2_std)
  return c_tot,c1_mean_val,c2_std_val

In [32]:
#### Initialization ###

ARB_VAL = 0.01
MEAN_FLUCT = 0.5
STD_TO_ARB = 3

tickers = list(pd.read_csv('/content/Nasdaq100_TickersOnly.csv')['Ticker'])
print(tickers)

start = '2020-09-20'
end = '2020-11-10'

SCAN_WIN_DAYS = 2

startDT = pd.to_datetime(start)
endDT = pd.to_datetime(end)

startDT = startDT.tz_localize(tz='America/New_York')
endDT = endDT.tz_localize(tz='America/New_York')

DEL_T = pd.Timedelta(days=SCAN_WIN_DAYS)
ONE_DAY = pd.Timedelta(days=1)
tmpDT = startDT

['ATVI', 'ADBE', 'AMD', 'ALXN', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AAL', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BMRN', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTXS', 'CTSH', 'CMCSA', 'CPRT', 'CSGP', 'COST', 'CSX', 'DLTR', 'EBAY', 'EA', 'EXC', 'EXPE', 'FB', 'FAST', 'FISV', 'FOXA', 'FOX', 'GILD', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KLAC', 'KHC', 'LRCX', 'LBTYA', 'LBTYK', 'LULU', 'MAR', 'MXIM', 'MELI', 'MCHP', 'MU', 'MSFT', 'MDLZ', 'MNST', 'NTAP', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'PCAR', 'PAYX', 'PYPL', 'PEP', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SBUX', 'SNPS', 'TMUS', 'TTWO', 'TSLA', 'TXN', 'TCOM', 'ULTA', 'UAL', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'WDC', 'WLTW', 'XEL', 'XLNX']


In [33]:
#### Find Best Stocks to Trade ###

# This ugly code should be fixed...
df = yf.download('MSFT', start , end ,interval='2m',progress=False)
days = sorted(set(df.index.date))
days = [x.strftime('%y-%m-%d') for x in days]
good_tcks = {i:[] for i in range(len(days))}

for tck in tickers:
  df = yf.download(tck, start , end ,interval='2m',progress=False)
  days = sorted(set(df.index.date))
  days = [x.strftime('%Y-%m-%d') for x in days]
  df['date'] = df.index
  df['date'] = df['date'].apply(lambda x:x.strftime('%Y-%m-%d') )
  # while (tmpDT + DEL_T <= endDT):
  for i in range(len(days)-SCAN_WIN_DAYS-1):
    rel_days = '|'.join(days[i:i+SCAN_WIN_DAYS])
    df_new = df[df['date'].str.contains(rel_days)]
    
    ### Find "good" stocks
    c_tot,c1_mean_val,c2_std_val = StockStatsCheck(df_new)    
    if (c_tot):
      print(tck,' ',days[i])
      good_tcks[i+SCAN_WIN_DAYS].append((tck, c1_mean_val, c2_std_val))


[ True] [ True  True]
ATVI   2020-09-30
[ True] [ True  True]
ATVI   2020-10-01
[ True] [ True  True]
ATVI   2020-10-02
[ True] [ True  True]
ATVI   2020-10-05
[ True] [ True  True]
ATVI   2020-10-12
[ True] [ True  True]
ATVI   2020-10-13
[ True] [ True  True]
ATVI   2020-10-19
[ True] [ True  True]
ATVI   2020-10-20
[ True] [ True  True]
ATVI   2020-10-21
[ True] [ True  True]
ATVI   2020-10-22
[ True] [ True  True]
ATVI   2020-10-23
[ True] [ True  True]
ATVI   2020-10-28
[ True] [ True  True]
ATVI   2020-10-29
[ True] [ True  True]
ATVI   2020-10-30
[ True] [ True  True]
ATVI   2020-11-02
[ True] [ True  True]
ATVI   2020-11-03
[ True] [ True  True]
ATVI   2020-11-04
[ True] [ True  True]
ADBE   2020-10-06
[ True] [ True  True]
ADBE   2020-10-09
[ True] [ True  True]
ADBE   2020-10-14
[ True] [ True  True]
ADBE   2020-10-15
[ True] [ True  True]
ADBE   2020-10-16
[ True] [ True  True]
ADBE   2020-10-19
[ True] [ True  True]
ADBE   2020-10-20
[ True] [ True  True]
ADBE   2020-10-21


In [75]:
#### Static Trade Simulation ###

profit = 1
days = sorted(days)

success_count=0;
fail_count = 0;

for i in range(len(good_tcks) - SCAN_WIN_DAYS - 1):
  if(good_tcks[i]!=[]):

    good_tcks_mean_c2=[]
    for j in range(len(good_tcks[i])):
      good_tcks_mean_c2.append(good_tcks[i][j][2].mean())

    good_tcks_mean_c2 = np.array(good_tcks_mean_c2)
    
    idx_to_trade = good_tcks_mean_c2.argmax()
    ticker_to_trade = good_tcks[i][idx_to_trade][0]
    
    df_today = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS-1] , days[i+SCAN_WIN_DAYS] ,interval='2m',progress=False)
    df_mean_today = (df_today['High']+df_today['Low'])/2
    mean_today_value = df_mean_today.median()
    buyPrice = mean_today_value *(1-ARB_VAL)
    sellPrice = buyPrice *(1+ARB_VAL)
    print(days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1])
    df_tommorow = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1] ,interval='2m',progress=False)

    
    ### intra-day dynamic trade check
    tommorow_highs = df_tommorow['High']
    tommorow_lows = df_tommorow['Low']

    buySuccess = (buyPrice > tommorow_lows)
    
    if (len(tommorow_lows[buySuccess])>0):
      firstBuyTime = tommorow_lows[buySuccess].index[0]
      sellSuccess = (sellPrice < tommorow_highs)
      if (len(tommorow_highs[sellSuccess])>0):
        lastSellTime = tommorow_highs[sellSuccess].index[-1]
        tradeSuccess = lastSellTime > firstBuyTime
      else:
        tradeSuccess = False
      
      if (tradeSuccess):
        profit *= 1 + (sellPrice - buyPrice)/buyPrice
        success_count+=1
      else:
        profit *= 0.995#1 + (df_tommorow['Low'][-1] - buyPrice)/buyPrice
        fail_count+=1




    #Static Deal Sim ####
    #simDayHigh, simDayLow = max(df_tommorow['High']), min(df_tommorow['Low'])
    #if simDayLow < buyPrice:
    #  if sellPrice < simDayHigh:
    #    profit *= 1 + (sellPrice - buyPrice)/buyPrice
    #    success_count+=1
    #  else:
    #    profit *= 0.995
    #    fail_count+=1
    
    
    
    print(days[i+SCAN_WIN_DAYS],' : p=',profit,'\n', ticker_to_trade, buyPrice, simDayLow)
    
print('')
print('total p=',profit)
print('success rate= ',success_count/(success_count+fail_count))

2020-10-02 2020-10-05
2020-10-02  : p= 1.01 
 AAL 12.38489975452423 198.5500030517578
2020-10-05 2020-10-06
2020-10-05  : p= 1.01 
 AAL 12.686850056648254 198.5500030517578
2020-10-06 2020-10-07
2020-10-06  : p= 1.0201 
 NFLX 509.1619427490234 198.5500030517578
2020-10-07 2020-10-08
2020-10-07  : p= 1.0149995 
 AAL 13.158981156349181 198.5500030517578
2020-10-08 2020-10-09
2020-10-08  : p= 1.025149495 
 AAL 12.914450941085816 198.5500030517578
2020-10-09 2020-10-12
2020-10-09  : p= 1.03540098995 
 AAL 12.954150037765503 198.5500030517578
2020-10-12 2020-10-13
2020-10-12  : p= 1.0302239850002501 
 AAL 13.129875283241272 198.5500030517578
2020-10-13 2020-10-14
2020-10-13  : p= 1.0302239850002501 
 ALXN 122.69317394256592 198.5500030517578
2020-10-14 2020-10-15
2020-10-14  : p= 1.0302239850002501 
 AAL 12.283425028324126 198.5500030517578
2020-10-15 2020-10-16
2020-10-15  : p= 1.025072865075249 
 EBAY 56.4448512840271 198.5500030517578
2020-10-16 2020-10-19
2020-10-16  : p= 1.019947500749

In [74]:
### Test with random stocks ###
profit = 1
success_count=0;
fail_count = 0;

for i in range(len(good_tcks) - SCAN_WIN_DAYS - 1):
  
  ticker_to_trade = random.choice(tickers)

  df_today = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS-1] , days[i+SCAN_WIN_DAYS] ,interval='2m',progress=False)
  df_mean_today = (df_today['High']+df_today['Low'])/2
  mean_today_value = df_mean_today.median()
  buyPrice = mean_today_value *(1-ARB_VAL)
  sellPrice = buyPrice *(1+ARB_VAL)
  print(days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1])
  df_tommorow = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1] ,interval='2m',progress=False)

  # intra-day dynamic trade check
  tommorow_highs = df_tommorow['High']
  tommorow_lows = df_tommorow['Low']

  buySuccess = (buyPrice > tommorow_lows)
  
  if (len(tommorow_lows[buySuccess])>0):
    firstBuyTime = tommorow_lows[buySuccess].index[0]
    sellSuccess = (sellPrice < tommorow_highs)
    if (len(tommorow_highs[sellSuccess])>0):
      lastSellTime = tommorow_highs[sellSuccess].index[-1]
      tradeSuccess = lastSellTime > firstBuyTime
    else:
      tradeSuccess = False
    
    if (tradeSuccess):
      profit *= 1 + (sellPrice - buyPrice)/buyPrice
      success_count+=1
    else:
      profit *= 0.995#1 + (df_tommorow['Low'][-1] - buyPrice)/buyPrice
      fail_count+=1

  print(days[i+SCAN_WIN_DAYS],' : p=',profit,'\n', ticker_to_trade, buyPrice, simDayLow)

print('')
print('total p=',profit)
print('success rate= ',success_count/(success_count+fail_count))


2020-09-30 2020-10-01
2020-09-30  : p= 1 
 QCOM 116.71605045318603 198.5500030517578
2020-10-01 2020-10-02
2020-10-01  : p= 1 
 GOOG 1466.4795556640624 198.5500030517578
2020-10-02 2020-10-05
2020-10-02  : p= 1 
 KHC 29.813849773406982 198.5500030517578
2020-10-05 2020-10-06
2020-10-05  : p= 1 
 MXIM 66.53789909362793 198.5500030517578
2020-10-06 2020-10-07
2020-10-06  : p= 1.01 
 CTAS 333.9344201660156 198.5500030517578
2020-10-07 2020-10-08
2020-10-07  : p= 1.00495 
 ORLY 446.2425 198.5500030517578
2020-10-08 2020-10-09
2020-10-08  : p= 1.00495 
 XLNX 104.61082454681396 198.5500030517578
2020-10-09 2020-10-12
2020-10-09  : p= 1.00495 
 VRTX 264.10477409362795 198.5500030517578
2020-10-12 2020-10-13
2020-10-12  : p= 1.00495 
 CSCO 39.50099962234497 198.5500030517578
2020-10-13 2020-10-14
2020-10-13  : p= 1.00495 
 MU 49.97025037765503 198.5500030517578
2020-10-14 2020-10-15
2020-10-14  : p= 0.99992525 
 DLTR 95.99039909362793 198.5500030517578
2020-10-15 2020-10-16
2020-10-15  : p= 1.

In [37]:
profit=1
idx_to_trade = good_tcks_mean_c2.argmax()
ticker_to_trade = good_tcks[i][idx_to_trade][0]

df_today = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS-1] , days[i+SCAN_WIN_DAYS] ,interval='2m',progress=False)
df_mean_today = (df_today['High']+df_today['Low'])/2
mean_today_value = df_mean_today.median()
buyPrice = mean_today_value *(1-ARB_VAL)
sellPrice = buyPrice *(1+ARB_VAL)
print(days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1])
df_tommorow = yf.download(ticker_to_trade, days[i+SCAN_WIN_DAYS] , days[i+SCAN_WIN_DAYS+1] ,interval='2m',progress=False)

simDayHigh, simDayLow = max(df_tommorow['High']), min(df_tommorow['Low'])





if simDayLow < buyPrice:
  if sellPrice < simDayHigh:
    profit *= 1 + (sellPrice - buyPrice)/buyPrice
    success_count+=1
  else:
    profit *= 0.995
    fail_count+=1
print(days[i+SCAN_WIN_DAYS],' : p=',profit,'\n', ticker_to_trade, buyPrice, simDayLow)

2020-11-06 2020-11-09
2020-11-06  : p= 1.01 
 PYPL 200.1383960723877 198.5500030517578


In [65]:
# semi-dynamic check
tommorow_highs = df_tommorow['High']
tommorow_lows = df_tommorow['Low']

buySuccess = (buyPrice > tommorow_lows)
print(buySuccess)
print(tommorow_lows[buySuccess])
firstBuyTime = tommorow_lows[buySuccess].index[0]
print('First Buy Time = ' , firstBuyTime)

sellSuccess = (sellPrice < tommorow_highs)
print(sellSuccess)
print(tommorow_highs[sellSuccess])
lastSellTime = tommorow_highs[sellSuccess].index[-1]
print('Last Sell Time = ' , lastSellTime)
dealSuccess = lastSellTime > firstBuyTime
print(dealSuccess)

Datetime
2020-11-06 09:30:00-05:00    False
2020-11-06 09:32:00-05:00    False
2020-11-06 09:34:00-05:00    False
2020-11-06 09:36:00-05:00    False
2020-11-06 09:38:00-05:00    False
                             ...  
2020-11-06 15:50:00-05:00    False
2020-11-06 15:52:00-05:00    False
2020-11-06 15:54:00-05:00    False
2020-11-06 15:56:00-05:00    False
2020-11-06 15:58:00-05:00    False
Name: Low, Length: 195, dtype: bool
Datetime
2020-11-06 09:42:00-05:00    200.000000
2020-11-06 09:48:00-05:00    199.960007
2020-11-06 09:50:00-05:00    198.979996
2020-11-06 09:52:00-05:00    199.020096
2020-11-06 09:54:00-05:00    198.645004
2020-11-06 09:56:00-05:00    198.809998
2020-11-06 09:58:00-05:00    198.550003
2020-11-06 10:00:00-05:00    198.800003
2020-11-06 10:02:00-05:00    199.270004
2020-11-06 10:04:00-05:00    199.160004
2020-11-06 10:06:00-05:00    200.070007
2020-11-06 10:08:00-05:00    199.934998
Name: Low, dtype: float64
First Buy Time =  2020-11-06 09:42:00-05:00
Datetime
20